# Multi-Model Multi-Stream Inference with DeGirum PySDK
This notebook example show how to build multi-model multi-stream apps using DeGirum PySDK and degirum_tools. <br>
Three common patterns are: <br>

    -3 models, 3 video streams — each in its own thread.
    -3 models, 1 video stream — fuse results with a compound model and a manual fusion variant.
    -3 models, 3 video streams — iterate results together in one loop (single‑threaded).

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

In [ ]:
from degirum_tools import ModelSpec  # Adjust import if needed
from degirum_tools import remote_assets

**ModelSpec**: One place to define the models (Declare models once and load them consistently, also keep device/runtime details in model_properties).
This example uses DeGirum’s Hailo zoo by default. Swap model names/zoo to match your project as needed.

In [ ]:
# === Specify where to run the inference ===
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
hw_location = "@local"
model_zoo_url = "degirum/hailo"

# === Sources (define once, reuse everywhere) ===
src1 = 0                                  # webcam (or your device index)
src2 = remote_assets.person_face_hand     # sample clip / replace with your path/URL
src3 = remote_assets.person_face_hand     # another source (replace as needed)

# === Model specs ===
model1_spec = ModelSpec(
    model_name="yolov8n_relu6_face--640x640_quant_hailort_multidevice_1",
    zoo_url=model_zoo_url,
    inference_host_address=hw_location,
    model_properties={"device_type": ["HAILORT/HAILO8L"]},
)

model2_spec = ModelSpec(
    model_name="yolov8n_relu6_hand--640x640_quant_hailort_multidevice_1",
    zoo_url=model_zoo_url,
    inference_host_address=hw_location,
    model_properties={"device_type": ["HAILORT/HAILO8L"]},
)

model3_spec = ModelSpec(
    model_name="yolov8n_relu6_person--640x640_quant_hailort_multidevice_1",
    zoo_url=model_zoo_url,
    inference_host_address=hw_location,
    model_properties={"device_type": ["HAILORT/HAILO8L"]},
)

# === Load model objects from specs (simple) ===
model1 = model1_spec.load_model()
model2 = model2_spec.load_model()
model3 = model3_spec.load_model()

## Use Case 1: 3 models, 3 video streams (each in a separate thread)

In [ ]:
import threading
import degirum_tools

import cv2
# Map models to sources and labels
configurations = [
    {"model": model1, "source": src1, "display_name": "Model 1 (Face)"},
    {"model": model2, "source": src2, "display_name": "Model 2 (Hand)"},
    {"model": model3, "source": src3, "display_name": "Model 3 (Person)"},
]

# Single-stream runner
def run_inference(model, source, display_name):
    with degirum_tools.Display(display_name) as output_display:
        for inference_result in degirum_tools.predict_stream(model, source):
            output_display.show(inference_result)
    print(f"✅ Stream '{display_name}' has finished.")

# Launch independent threads
threads = []
for cfg in configurations:
    t = threading.Thread(
        target=run_inference,
        args=(cfg["model"], cfg["source"], cfg["display_name"]),
        daemon=True,
    )
    threads.append(t)
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()

print("🎉 All inference streams have been processed.")

[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] \ InferenceResults.__init__ 1153.0us 
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ DetectionResults.__init__ 5484.2us 
[ DEBUG ] \ Model._create_postprocessor 8697.6us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 15698.5us 
[ DEBUG ] \ _Preprocessor_CV.forward 4998.400000000001us 
[ DEBUG ] \ _PreprocessorAggregator.forward 8449.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #7
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ INFO ] Received frame #6 results
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _CloudServerModel._predict_handler.<locals

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 9506.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 25133.0us 
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 25740.3us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 35467.9us 
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 25740.3us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] \ InferenceResults.__init__ 1016.9us 
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] \ DetectionResults.__init__ 7244.6us 
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ Model._create_postprocessor 7244.6us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] \ InferenceResults.__init__ 7141.900000000001us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 17080.9us 
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBU

[ DEBUG ] \ Model._create_postprocessor 12826.800000000001us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 12826.800000000001us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 4372.2us 
[ DEBUG ] \ _PreprocessorAggregator.forward 4372.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] \ _Preprocessor_CV.forward 5262.900000000001us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ DEBUG ] \ _PreprocessorAggregator.forward 5909.900000000001us 
[ INFO ] sending frame #17
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ INFO ] sending frame #13
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 14207.7us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 14207.7us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] \ InferenceResults.__init__ 9441.800000000001us 
[ DEBUG ] \ DetectionResults.__init__ 9441.800000000001us 
[ DEBUG ] \ Model._create_postprocessor 9441.800000000001us 
[ INFO ] Received frame #16 results
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 15110.7us 
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 1867.3us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 9551.4us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preproc

[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 12535.300000000001us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 0.0us 
[ DEBUG ] \ _PreprocessorAggregator.forward 8049.3us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #23
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 2014.4us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 6050.8us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 6050.8us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 10351.7us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] / _Clou

...

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 2992.3us 
[ INFO ] Received frame #169 results
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _PreprocessorAggregator.forward 11615.6us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 11615.6us 
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 9606.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ INFO ] sending frame #90
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 13414.2us 
[ INFO ] sending frame #142
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ DEBUG ] / _CloudServerModel._Clou

[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 5008.900000000001us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] \ _Preprocessor_CV.forward 7920.0us 
[ DEBUG ] \ _Preprocessor_CV.forward 8920.300000000001us 
[ INFO ] Received frame #91 results
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 12921.9us 
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _PreprocessorAggregator.forward 10919.1us 
[ DEBUG ] \ _PreprocessorAggregator.forward 10919.1us 
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ INFO ] Received frame #171 results
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.call

[ DEBUG ] \ DetectionResults.__init__ 10215.5us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] \ Model._create_postprocessor 13364.300000000001us 
[ DEBUG ] \ Model._create_postprocessor 13364.300000000001us 
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 17731.600000000002us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 17731.600000000002us 
[ INFO ] Received frame #145 results
[ DEBUG ] \ _Preprocessor_CV.forward 0.0us 
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _PreprocessorAggregator.forward 12941.7us 
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 9791.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__

KeyboardInterrupt: 

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync


## Use Case 2: 3 models, 1 video stream (combine results)
Two options here:

    A) **Compound model** (simplest) - Let the tooling fuse results for you using CombiningCompoundModel. <br>
    B) **Manual fusion** (more control) - Run three predictors off the same video stream and merge results yourself.

### Compound model (simplest)

In [ ]:
import degirum_tools

# Use the first source for the single-stream case
video_source = src1

# Compose a compound model from your three models
combined_model = degirum_tools.CombiningCompoundModel(
    degirum_tools.CombiningCompoundModel(model2, model1),
    model3,
)

# Stream + display
with degirum_tools.Display("Compound: Models 1+2+3") as display:
    for inference_result in degirum_tools.predict_stream(combined_model, video_source):
        display.show(inference_result)

[ INFO ] Received frame #4 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 493.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 4773.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect
[ INFO ] websocket connection successful
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect 6209.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 134343.3us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_connect 137344.9us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.reconnect 138342.1us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__ 147468.2us 
[ DEBUG ] \ _CloudServerModel._init_runtime 150919.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.set_callback
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.set_callback 2006.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.st

[ INFO ] Received frame #2 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 0.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect
[ INFO ] websocket connection successful
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect 5120.900000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 108319.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_connect 109341.40000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.reconnect 112465.90000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__ 131132.9us 
[ DEBUG ] \ _CloudServerModel._init_runtime 135549.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.set_callback
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.set_callback 1595.1000000000001us 
[ DEBUG ] / _C

[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 0.0us 
[ DEBUG ] \ Model._create_postprocessor 0.0us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 0.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] \ _Preprocessor_CV.forward 10392.4us 
[ INFO ] Received frame #3 results
[ DEBUG ] \ _PreprocessorAggregator.forward 12402.800000000001us 
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 1996.8us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<l

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 3271.4us 
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 5296.7us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 0.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 0.0us 
[ DEBUG ] \ _PreprocessorAggregator.forward 811.7us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #10
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 15291.800000000001us 
[ DEBUG ] \ _CloudServerModel._CloudSer

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 12501.9us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 13033.1us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 0.0us 
[ DEBUG ] \ _PreprocessorAggregator.forward 0.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #12
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG 

...

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 1872.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #5
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 13325.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 15018.800000000001us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #6
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 973.1us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending 

[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #2
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 1719.9us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 1719.9us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 2720.3us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #3
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #4
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _Cl

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 3152.1us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 7733.5us 
[ DEBUG ] \ DetectionResults.__init__ 7733.5us 
[ DEBUG ] \ Model._create_postprocessor 8745.300000000001us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 8745.300000000001us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Received frame #5 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _Preprocessor_CV.forward 4002.2000000000003us 
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _PreprocessorAggregator.forward 4002.2000000000003us 
[ 

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 2262.8us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 4268.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #1
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #2
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG ] / _CloudServerModel.

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_disconnect 11837.300000000001us 
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 3010.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.disconnect 19603.100000000002us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 15886.800000000001us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__del__
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.disconnect
[ INFO ] Received frame #6 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.set_callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.set_callback 2007.8us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 4653.3us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.disconnect 4653.3u

### Manual fusion (more control)

In [ ]:
import degirum_tools
from itertools import zip_longest

with degirum_tools.Display("Manual Fusion (Single Stream)") as display, \
     degirum_tools.open_video_stream(src1) as video_stream:

    # Create prediction generators bound to the same underlying stream
    p1 = model1.predict_batch(degirum_tools.video_source(video_stream))
    p2 = model2.predict_batch(degirum_tools.video_source(video_stream))
    p3 = model3.predict_batch(degirum_tools.video_source(video_stream))

    # Iterate in lockstep; guard against None frames
    for r1, r2, r3 in zip_longest(p1, p2, p3):
        if r1 is None or r2 is None or r3 is None:
            continue

        # Merge detections into one result; reuse r1 as the carrier
        r1.results.extend(r2.results)
        r1.results.extend(r3.results)

        display.show(r1.image_overlay)

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 1459.9us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 10512.2us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 1302.8us 
[ DEBUG ] \ _PreprocessorAggregator.forward 2255.1us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #5
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__del__
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.disconnect
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.set_callback
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.set_callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.disconnect 13492.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.stop_async_thread
[ DEBUG ] \ _Cloud

[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 0.0us 
[ DEBUG ] \ Model._create_postprocessor 1512.2us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 2521.1us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 6013.2us 
[ DEBUG ] \ _PreprocessorAggregator.forward 8055.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #1
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG ] \ _CloudServ

[ INFO ] sending frame #8
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 6430.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 6430.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 6430.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 8440.9us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 2430.7000000000003us 
[ DEBUG ] \ Model._create_postprocessor 2430.7000000000003us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 2430.7000000000003us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 9540.2us 
[ DEBUG ] \ _Preproces

[ DEBUG ] \ _Preprocessor_CV.forward 1683.5us 
[ DEBUG ] \ _PreprocessorAggregator.forward 1683.5us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #10
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 9409.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 9409.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 9409.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 9409.0us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 5833.400000000001us 
[ DEBUG ] \ DetectionResults.__init__ 5833.400000000001us 
[ DEBUG ] \ Model._create_postprocessor 5833.400000000001

[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #12
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 6824.900000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 6824.900000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 6824.900000000001us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 953.5us 
[ DEBUG ] \ DetectionResults.__init__ 8716.1us 
[ DEBUG ] \ Model._create_postprocessor 8716.1us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 8716.1us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Receive

...

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 2968.8us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 4002.2000000000003us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 5001.6us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 0.0us 
[ DEBUG ] \ Model._create_postprocessor 4343.8us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 4343.8us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 3795.9us 
[ DEBUG ] \ _PreprocessorAggregator.forward 3795.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #58
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _

[ DEBUG ] \ Model._create_postprocessor 7343.2us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 9357.7us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 1961.9us 
[ DEBUG ] \ _PreprocessorAggregator.forward 2905.6us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #66
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 0.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 2990.3us 
[ DEBUG ] \ _PreprocessorAggregato

[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #0
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 1011.4us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 3665.9us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 6655.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #1
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 1263.2us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 3967.2000000000003us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 5984.3us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #2
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_wait 1008980.8us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_connect
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect
[ INFO ] websocket connection successful
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect 960.8000000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 97463.1us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_connect 99413.7us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #0
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 2271.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 2636.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 5662.7us 
[ DEBUG ] / _Clo

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Received frame #6 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 0.0us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Received frame #7 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 1998.7us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Received frame #8 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 0.0us 
[ DE

## Use Case 3: 3 models, 3 video streams (iterated together in a single thread)

In [ ]:
import degirum_tools
from itertools import zip_longest

# Use a separate display per stream
with degirum_tools.Display("Model 1 (src1)") as d1, \
     degirum_tools.Display("Model 2 (src2)") as d2, \
     degirum_tools.Display("Model 3 (src3)") as d3, \
     degirum_tools.open_video_stream(src1) as s1, \
     degirum_tools.open_video_stream(src2) as s2, \
     degirum_tools.open_video_stream(src3) as s3:

    # Create prediction generators
    p1 = model1.predict_batch(degirum_tools.video_source(s1))
    p2 = model2.predict_batch(degirum_tools.video_source(s2))
    p3 = model3.predict_batch(degirum_tools.video_source(s3))

    # Advance all three streams in lockstep
    for r1, r2, r3 in zip_longest(p1, p2, p3):
        if r1 is not None:
            d1.show(r1)
        if r2 is not None:
            d2.show(r2)
        if r3 is not None:
            d3.show(r3)

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect
[ INFO ] websocket connection successful
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect 9431.800000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 111546.2us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_connect 112558.90000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.reconnect 117940.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__ 124642.5us 
[ DEBUG ] \ _CloudServerModel._init_runtime 126784.1us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.set_callback
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.set_callback 996.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.start_run
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.start_run 931.2us 
[ DEBUG ] / Model.predict_batch.<locals>.source
[ DEBUG ] \ Model.predict_batch.<locals>.source 0.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Pre

[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 1872.6000000000001us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_connect
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 6415.8us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect
[ INFO ] websocket connection successful
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.connect 7952.1us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 109903.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_connect 112915.6us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.reconnect 119899.40000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__ 143470.5us 
[ DEBUG ] \ _CloudServerModel._init_runtime 145612.1us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.set_callback
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.set_callback 0.0us 
[ DEBU

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 4789.400000000001us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 8497.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 14208.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 15207.7us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 0.0us 
[ DEBUG ] \ Model._create_postprocessor 0.0us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 5293.900000000001us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 2999.1us 
[ DEBUG ] \ _PreprocessorAggregator.forward 4000.4us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _

[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 3098.2000000000003us 
[ DEBUG ] \ _Preprocessor_CV.forward 1099.5us 
[ DEBUG ] \ _PreprocessorAggregator.forward 2101.2us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #9
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 1439.3us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 3116.5us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 5127.3us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 16294.6us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._

[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 0.0us 
[ DEBUG ] \ Model._create_postprocessor 0.0us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 0.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Received frame #5 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _Preprocessor_CV.forward 4012.3us 
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 1003.5us 
[ DEBUG ] \ _PreprocessorAggregator.forward 9539.7us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 4248.1us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime

[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #8
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 3207.8us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio_emit 7381.1us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime._send_data 8381.1us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.predict 8833.5us 
[ DEBUG ] / Model._predict_impl.<locals>.prepare_results
[ DEBUG ] / Model._create_postprocessor
[ DEBUG ] / DetectionResults.__init__
[ DEBUG ] / InferenceResults.__init__
[ DEBUG ] \ InferenceResults.__init__ 0.0us 
[ DEBUG ] \ DetectionResults.__init__ 0.0us 
[ DEBUG ] \ Model._create_postprocessor 0.0us 
[ DEBUG ] \ Model._predict_impl.<locals>.prepare_results 0.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.for

[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 11618.9us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.__init__.<locals>.results
[ INFO ] Received frame #14 results
[ DEBUG ] / _CloudServerModel._predict_handler.<locals>.callback
[ DEBUG ] \ _CloudServerModel._predict_handler.<locals>.callback 3545.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.__init__.<locals>.results 3545.0us 
[ DEBUG ] / _PreprocessorAggregator.forward
[ DEBUG ] / _Preprocessor_CV.forward
[ DEBUG ] \ _Preprocessor_CV.forward 4363.3us 
[ DEBUG ] \ _PreprocessorAggregator.forward 4363.3us 
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.predict
[ DEBUG ] / _CloudServerModel._CloudServerRuntime._send_data
[ INFO ] sending frame #10
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.sio_emit
[ DEBUG ] / _CloudServerModel._CloudServerRuntime.run_coro_sync
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.run_coro_sync 0.0us 
[ DEBUG ] \ _CloudServerModel._CloudServerRuntime.sio